In [ ]:
from langchain_community.vectorstores import Chroma

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

import pandas as pd
import yaml
from pprint import pprint

In [ ]:
OPENAI_API_KEY = yaml.safe_load(open('../credentials.yml'))['openai']

In [ ]:
embedding_function = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    api_key=OPENAI_API_KEY
)

In [ ]:
vectorstore = Chroma(
    persist_directory="data/chroma.db",
    embedding_function=embedding_function
)

In [ ]:
retriever = vectorstore.as_retriever()

retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f87893e6020>)

In [ ]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f87893e6020>)

In [ ]:
# * Prompt template 

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [ ]:
model = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.7,
    api_key=OPENAI_API_KEY
)

In [ ]:
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f878990eaa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f878990f940>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f87893e6020>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f878990eaa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f878990f940>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [ ]:
result = rag_chain.invoke("What are the top 3 things needed in a good social media marketing strategy?")

In [ ]:
result

'1. Setting clear and specific goals related to business objectives.\n2. Choosing the right social media platforms where your target audience is present.\n3. Developing a content calendar and posting schedule to maintain consistency and engagement with the audience.'

In [ ]:
pprint(result)

('1. Setting clear and specific goals related to business objectives.\n'
 '2. Choosing the right social media platforms where your target audience is '
 'present.\n'
 '3. Developing a content calendar and posting schedule to maintain '
 'consistency and engagement with the audience.')
